In [1]:
import sys
sys.path.append("..")
from ecoscape_layers.layers import generate_layers

In [2]:
REDLIST_KEY = input("IUCN Red List API key: ")
EBIRD_KEY = input("EBird API key: ")

species_list_path = "./inputs/test_species_list.txt"
terrain_path = "./inputs/test_terrain.tif"
terrain_codes_path = "./inputs/terrain_codes.csv"
species_range_folder = "./inputs/ebird_ranges"
output_folder = "./outputs"

crs = 'PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
resolution = 300
resampling = "near"
bounds = (-2214130.5469038677401841, 2459449.2663429058156908, -2174930.5469038677401841, 2504349.2663429058156908)
padding = 10000

refine_method = "forest"

In [3]:
generate_layers(REDLIST_KEY, EBIRD_KEY, species_list_path, terrain_path, terrain_codes_path, species_range_folder,
                    output_folder, crs, resolution, resampling, bounds, padding, refine_method)

Generating terrain layer...
Gathering species habitat preferences...
Creating resistance CSVs...
Generating habitat layers...
Layers successfully generated in ./outputs
